In [9]:
import matplotlib
import numpy as np
import matplotlib.pyplot as plt

# matplotlib.use('QtAgg')
%matplotlib notebook

# Optimizing a stick-zeppelin tissue model

## Defining the tissue model

In [10]:
# components from dmipy that are required to define the tissuemodel outlined above
from dmipy.core.modeling_framework import MultiCompartmentModel
from dmipy.signal_models import cylinder_models, gaussian_models

# The wrapper from microtool
from microtool.dmipy import DmipyTissueModel

# Cylinder orientation angles theta, phi := mu
mu = np.array([np.pi / 2, np.pi/2])
# Parralel diffusivity lambda_par in E-9 m^2/s (in the paper d_par)
lambda_par = 1.7e-9
lambda_perp = 0.2e-9

zeppelin = gaussian_models.G2Zeppelin(mu, lambda_par, lambda_perp)
stick = cylinder_models.C1Stick(mu, lambda_par)
stick_zeppelin = MultiCompartmentModel(models=[zeppelin, stick])
# fixing the parralel diffusivities to each other (The stick diffusivity is removed from optimization)
stick_zeppelin.set_equal_parameter('G2Zeppelin_1_lambda_par', 'C1Stick_1_lambda_par')
tissue_model =  DmipyTissueModel(stick_zeppelin, volume_fractions=[0.5, 0.5])

print(tissue_model)

Tissue model with 10 scalar parameters:
Tissueparameter             Value    Scale  Optimize
------------------------  -------  -------  ----------
G2Zeppelin_1_mu_0         1.5708     1      True
G2Zeppelin_1_mu_1         1.5708     1      True
G2Zeppelin_1_lambda_par   1.7e-09    1e-09  True
G2Zeppelin_1_lambda_perp  2e-10      1e-09  True
C1Stick_1_mu_0            1.5708     1      True
C1Stick_1_mu_1            1.5708     1      True
C1Stick_1_lambda_par      1.7e-09    1e-09  False
partial_volume_0          0.5        1      True
partial_volume_1          0.5        1      True
S0                        1          1      False


## Defining the initial acquisition scheme

In [ ]:
from microtool.acquisition_scheme import DiffusionAcquisitionScheme
from microtool.utils.saved_schemes import alexander2008
from microtool.dmipy import convert_dmipy_scheme2diffusion_scheme
from microtool.gradient_sampling.uniform import sample_uniform
from microtool.utils.plotting import plot_acquisition_parameters



scheme = convert_dmipy_scheme2diffusion_scheme(alexander2008())
fig = plot_acquisition_parameters(scheme)
plt.tight_layout()
plt.show()

In [12]:
# fixing b values for b0 measurements since dmipy will freak out if you try to optimize these
scheme["DiffusionBValue"].set_fixed_mask(scheme.b_values == 0)

## The optimization procedure

In [ ]:
from microtool.optimize import optimize_scheme

# The variance of the assumed gaussian noise
noise_variance = 0.02

optimal_scheme, _ = optimize_scheme(scheme, tissue_model, noise_variance, method='trust-constr')

In [ ]:
plot_acquisition_parameters(optimal_scheme)
plt.tight_layout()

## Verification trough Monte Carlo simulations

In [ ]:
from microtool.monte_carlo import MonteCarloSimulation
from microtool.monte_carlo.parameter_distributions import plot_parameter_distributions
from scipy import stats

# Setting up noise distribution 
noise_dist = stats.norm(loc=0,scale=np.sqrt(noise_variance))

# making the monte carlo simulation
simulation = MonteCarloSimulation(scheme,tissue_model, noise_dist, n_sim = 100)

# disabling dmipy's parralel processing since it is actually slower if only one voxel is fitted
simulation.set_fitting_options({"use_parallel_processing":False})

result = simulation.run()


In [ ]:
plot_parameter_distributions(result,tissue_model)
plt.tight_layout()


In [ ]:
simulation.set_scheme(optimal_scheme)
optimal_result = simulation.run()


In [ ]:
plot_parameter_distributions(optimal_result, tissue_model)
plt.tight_layout()